<a href="https://www.kaggle.com/code/udayr007/geosaptialmaps?scriptVersionId=151770980" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import math
import geopandas as gpd

In [2]:
import folium
from folium import Marker, Choropleth, Circle
from folium.plugins import HeatMap, MarkerCluster

In [3]:
m_1 = folium.Map(location = [42.32, -71.0589], tiles = 'openstreetmap', zoom_start = 13)
m_1

In [4]:
crimes = pd.read_csv("../input/geospatial-learn-course-data/crimes-in-boston/crimes-in-boston/crime.csv", encoding='latin-1')


In [5]:
crimes.dropna(subset = ['Lat','Long','DISTRICT'], inplace = True)
crimes = crimes[crimes.YEAR >= 2018]
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [6]:
crimes[crimes.OFFENSE_CODE_GROUP == 'Robbery'].head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
8,I182070931,301,Robbery,ROBBERY - STREET,C6,177,NaN,2018-09-03 20:48:00,2018,9,Monday,20,Part One,MASSACHUSETTS AVE,42.331521,-71.070853,"(42.33152148, -71.07085307)"
130,I182070795,301,Robbery,ROBBERY - STREET,C11,362,NaN,2018-09-02 20:30:00,2018,9,Sunday,20,Part One,LYNDHURST ST,42.292906,-71.069558,"(42.29290563, -71.06955795)"
176,I182070741,301,Robbery,ROBBERY - STREET,D14,805,NaN,2018-09-03 03:26:00,2018,9,Monday,3,Part One,PORTSMOUTH ST,42.360533,-71.144263,"(42.36053257, -71.14426332)"
273,I182070621,361,Robbery,ROBBERY - OTHER,A7,23,NaN,2018-09-02 18:09:00,2018,9,Sunday,18,Part One,MARION ST,42.377256,-71.036674,"(42.37725644, -71.03667354)"
299,I182070598,311,Robbery,ROBBERY - COMMERCIAL,A15,60,NaN,2018-09-02 17:15:00,2018,9,Sunday,17,Part One,RUTHERFORD AVE,42.371673,-71.063264,"(42.37167264, -71.06326413)"


In [7]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == "Towed") &
                           (crimes.HOUR.isin(range(9,18))))]

In [8]:
crimes.OFFENSE_CODE_GROUP.unique()

array(['Larceny', 'Vandalism', 'Towed', 'Investigate Property',
       'Motor Vehicle Accident Response', 'Auto Theft', 'Verbal Disputes',
       'Robbery', 'Fire Related Reports', 'Other', 'Property Lost',
       'Assembly or Gathering Violations', 'Larceny From Motor Vehicle',
       'Medical Assistance', 'Residential Burglary', 'Simple Assault',
       'Restraining Order Violations', 'Violations', 'Harassment',
       'Ballistics', 'Property Found', 'Police Service Incidents',
       'Disorderly Conduct', 'Property Related Damage',
       'Missing Person Reported', 'Investigate Person', 'Fraud',
       'Drug Violation', 'Aggravated Assault',
       'License Plate Related Incidents', 'Firearm Violations',
       'Other Burglary', 'Arson', 'Warrant Arrests', 'Bomb Hoax',
       'Counterfeiting', 'Liquor Violation', 'Firearm Discovery',
       'Landlord/Tenant Disputes', 'Missing Person Located',
       'Auto Theft Recovery', 'Service', 'Operating Under the Influence',
       'Confiden

In [9]:
daytime_robberies.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
93,I182070832,3410,Towed,TOWED MOTOR VEHICLE,C6,229,NaN,2018-09-03 13:18:00,2018,9,Monday,13,Part Three,E SIXTH ST,42.332987,-71.042824,"(42.33298734, -71.04282382)"
137,I182070787,3410,Towed,TOWED MOTOR VEHICLE,E18,499,NaN,2018-09-03 10:11:00,2018,9,Monday,10,Part Three,NAVARRE ST,42.277589,-71.117552,"(42.27758926, -71.11755202)"
151,I182070776,3410,Towed,TOWED MOTOR VEHICLE,D14,794,NaN,2018-09-03 10:08:00,2018,9,Monday,10,Part Three,GREYLOCK RD,42.350427,-71.134961,"(42.35042708, -71.13496065)"
293,I182070605,3410,Towed,TOWED MOTOR VEHICLE,D14,793,NaN,2018-09-02 17:20:00,2018,9,Sunday,17,Part Three,COMMONWEALTH AVE,42.348889,-71.133545,"(42.34888872, -71.13354530)"
320,I182070575,3410,Towed,TOWED MOTOR VEHICLE,D14,816,NaN,2018-09-02 14:07:00,2018,9,Sunday,14,Part Three,ROYAL ST,42.357511,-71.129965,"(42.35751060, -71.12996479)"


In [10]:
m_2 = folium.Map(location = [42.32, -71.0589],zoom_start = 13,tiles='cartodbpositron')

for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2)

m_2

In [11]:
m_2 = folium.Map(location = [42.32, -71.0589],zoom_start = 13, tiles = 'OpenStreetMap')

for idx, row in daytime_robberies.iterrows():
    Marker((row['Lat'], row['Long'])).add_to(m_2)

m_2

In [12]:
m_3 = folium.Map(location = [42.32, -71.0589], zoom_start = 13, tiles = 'cartodbpositron')

mc = MarkerCluster()